In [31]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import DensePassageRetriever
from haystack.pipelines import Pipeline
from haystack.nodes import TransformersReader
from haystack.utils import print_answers

document_store = FAISSDocumentStore.load("document_store.faiss")

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True
)
reader = TransformersReader(model_name_or_path="..\initial_moddeling\distilbert-nlb-qa", use_gpu=True)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


In [32]:
p_extractive = Pipeline()
p_extractive.add_node(component=retriever, name="Retriever", inputs=["Query"])
p_extractive.add_node(component=reader, name="Reader", inputs=["Retriever"])
res = p_extractive.run(
    query="When were the anticorruption policies and procedures revised?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)
print_answers(res, details="minimum")


'Query: When were the anticorruption policies and procedures revised?'
'Answers:'
[   {   'answer': '2020',
        'context': 'ules, instructions, or\n'
                   'There were no confirmed cases of corruption in 2020. '},
    {   'answer': '2020',
        'context': 'and Bribery\n'
                   'and on Management of Conflicts of Interest was renewed in '
                   '2020. The\n'
                   'system of risk management of corruption and bribery was '
                   'suppleme'},
    {   'answer': 'second\nhalf of 2020',
        'context': 'o corruption\n'
                   'The anticorruption policy and\n'
                   'procedures were revised in second\n'
                   'half of 2020. There were no confirmed\n'
                   'cases of corruption in 2020.\n'
                   'Non-discriminat'},
    {   'answer': '2021',
        'context': 'Implementation activities will continue in 2021.\n'
                   'The main risks in relation 